In [1]:
# %pwd

'd:\\AI_ChatBot\\research'

In [ ]:
# import os
# os.chdir("../")

In [3]:
%pwd

'd:\\AI_ChatBot'

In [19]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
#extract data from the pdf
def load_pdf_file(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader)
    
    documents=loader.load()
    return documents

In [21]:
extracted_data=load_pdf_file(data='Data/')

In [22]:
# split the data into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [23]:
text_chunks=text_split(extracted_data)
print(len(text_chunks))

2371


In [25]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [26]:
# download the embedding from hugging face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [28]:
embeddings = download_hugging_face_embeddings()

In [30]:
query_result = embeddings.embed_query('hi chandima')
print(query_result)

[0.015530286356806755, 0.14416012167930603, -0.003489758586511016, 0.04087614268064499, -0.058465588837862015, 0.029729219153523445, 0.05767727643251419, -0.004789259750396013, -0.017884263768792152, -0.031429361552000046, 0.08618224412202835, -0.029026849195361137, -0.03918154910206795, -0.03882989287376404, 0.058014724403619766, -0.003082044655457139, -0.0811605453491211, -0.011851095594465733, -0.09150673449039459, -0.05118604004383087, 0.1090053915977478, 0.02310873754322529, 0.009349126368761063, 0.0028737829998135567, -0.06266666948795319, -0.06153743341565132, 0.09525725990533829, -0.018649205565452576, 0.0822243019938469, -0.06310760974884033, -0.008218184113502502, 0.06347844004631042, 0.0397406667470932, -0.0024888531770557165, -0.042481016367673874, 0.1324295550584793, 0.042675700038671494, -0.002224695635959506, 0.07782162725925446, -0.030601365491747856, 0.00992884486913681, -0.03769165277481079, 0.027743985876441002, 0.030646437779068947, -0.013506635092198849, -0.0558970

In [50]:
from dotenv import load_dotenv
load_dotenv()

True

In [51]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [33]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "aichatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1",
    )
)

{
    "name": "aichatbot",
    "metric": "cosine",
    "host": "aichatbot-ano5e1g.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [52]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [36]:
#embed each chunks into pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
#load existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [38]:
docsearch

In [39]:
retriever =  docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [44]:
retrieved_docs = retriever.invoke("what is Hidden Markov Models?")

In [45]:
retrieved_docs

[Document(id='0b071a65-2737-4380-a623-10f8df17584f', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2024-09-19T22:03:01+00:00', 'page': 74.0, 'page_label': '75', 'producer': 'Pdftools SDK', 'source': 'Data\\Artificial Intelligence.pdf', 'total_pages': 533.0}, page_content='3.3 Hidden Markov Models  Hidden Markov Models (HMMs) are extremely common in the ML analysis of sequential data that has a dependence on time. Typically, a model will be able to explain a sequence and predict or generate new sequences. The techniques are most at home with Natural Language Processing (NLP) and with handwriting recognition, speech recognition, and biological informatics.  With an HMM there are two collections of probabilities. In the background, there is a probabilistic Markov'),
 Document(id='bbfbdd0d-66db-4767-98fb-6fe8c02e8e4a', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2024-09-19T22:03:01+00:00', 'page': 488.0, 'page_label': '489', 'producer': 'Pdftools SDK', 'so

In [54]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="qwen-qwq-32b")

In [61]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a highly knowledgeable and concise AI assistant specialized in answering complex questions "
    "using the provided context. Carefully analyze the retrieved context to construct an accurate and helpful response.\n\n"
    "Guidelines:\n"
    "- Base your answer strictly on the retrieved context below. Do not rely on prior knowledge.\n"
    "- If the context does not contain enough information, respond with: 'I don't know based on the provided context.'\n"
    "- Limit your response to a maximum of **three well-structured sentences**.\n"
    "- Be clear, precise, and avoid unnecessary elaboration.\n\n"
    "Context:\n{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [62]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [69]:
response = rag_chain.invoke({"input": "what is the dog"})
print(response["answer"])


<think>
Okay, let's tackle the question "what is the dog" based on the provided context. First, I need to check the context given. The user provided a lengthy text about self-supervised learning, examples like spam detection, dog breeds (poodle, beagle, pug), and references to how context and previous dialogues help in determining answers. The example with Dan Bobrow shows that sometimes answers are references, not statements.

The question is straightforward: "what is the dog". The context mentions that in a model identifying dog breeds, the classes could be poodle, beagle, pug, etc. However, the context doesn't provide any specific information about the dog in question. The example given relates to a reservation name, but that's not about dogs. Since there's no further details in the context about a specific dog or its description, I can't determine an answer from the provided text. The user might be expecting a breed, but without more context, the answer isn't here.
</think>

I don